# Get ground truths for test set

Method:
1. Grab all data from the original dataset from https://archive.ics.uci.edu/ml/datasets/Adult
2. For each record in test.csv:
> Find a record in the original dataset that matches all the fields in the record
> Save the exceeds50K result

Conclusion:</br>
Prediction accuracy results based on this set of ground truth does not completely match results generated by Kaggle submissions.

In [1]:
# imports

# data manipulation
import pandas as pd
import numpy as np

# visualiation
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
# load test data
test_file = './data/test.csv'
test_data = pd.read_csv(test_file)
test_data = pd.DataFrame(data=test_data)
print(test_data.dtypes)
test_data.head()

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
dtype: object


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country
0,23,Private,32732,Some-college,10,Married-civ-spouse,Sales,Husband,Male,0,0,25,United-States
1,69,Private,165017,HS-grad,9,Widowed,Machine-op-inspct,Unmarried,Male,2538,0,40,United-States
2,27,Private,36440,Bachelors,13,Never-married,Sales,Not-in-family,Female,0,0,40,United-States
3,40,Private,182217,Some-college,10,Married-civ-spouse,Other-service,Wife,Female,0,0,40,Scotland
4,24,Private,89347,11th,7,Never-married,Machine-op-inspct,Not-in-family,Female,0,0,40,United-States


In [33]:
# load original data
folder = './original_data/'
original_train_data_file = folder+'adult.data'
original_train_data = pd.read_csv(original_train_data_file)

original_test_data_file = folder+'adult.test'
original_test_data = pd.read_csv(original_test_data_file)

original_data = pd.concat([original_train_data, original_test_data])
original_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,exceeds50K
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [34]:
# match labels
test_ground_truths = []
for index, row in test_data.iterrows():
    found_target = original_data[(original_data['age'] == row['age']) & \
                                               (original_data['workclass'] == row['workclass']) & \
                                               (original_data['fnlwgt'] == row['fnlwgt']) & \
                                               (original_data['education-num'] == row['education-num']) & \
                                               (original_data['marital-status'] == row['marital-status']) & \
                                               (original_data['occupation'] == row['occupation']) & \
                                               (original_data['relationship'] == row['relationship']) & \
                                               (original_data['sex'] == row['sex']) & \
                                               (original_data['capital-gain'] == row['capital-gain']) & \
                                               (original_data['capital-loss'] == row['capital-loss']) & \
                                               (original_data['hours-per-week'] == row['hours-per-week']) & \
                                               (original_data['native-country'] == row['native-country'])\
                                              ]
    if(found_target.shape[0] != 1):
        resultshouldbe = found_target.iloc[0].exceeds50K
        for target_index, target_row in found_target.iterrows():
            if(target_row['exceeds50K'] != resultshouldbe):
                print("error")
                print(index)
                print(found_target)
                break;
    test_ground_truths.append(found_target.iloc[0].exceeds50K)


error
4216
       age workclass  fnlwgt   education  education-num       marital-status  \
848     39   Private  138192   Bachelors             13   Married-civ-spouse   
22761   39   Private  138192   Bachelors             13   Married-civ-spouse   

          occupation relationship    race    sex  capital-gain  capital-loss  \
848     Craft-repair      Husband   White   Male             0             0   
22761   Craft-repair      Husband   White   Male             0             0   

       hours-per-week  native-country exceeds50K  
848                40   United-States      <=50K  
22761              40   United-States       >50K  
error
6188
       age workclass  fnlwgt education  education-num  marital-status  \
29506   34   Private  169583   Masters             14   Never-married   
2373    34   Private  169583   Masters             14   Never-married   

             occupation    relationship    race      sex  capital-gain  \
29506   Exec-managerial   Not-in-family   White  

In [40]:
len(test_ground_truths)
output = "exceeds50K\n"
for ground_truth in test_ground_truths:
    output += '1\n' if ground_truth == '>50K' else '0\n'
    
ground_truths_file = open('./data/test_ground_truths.csv', 'w')
ground_truths_file.write(output)
ground_truths_file.close()